In [1]:
import sys
import os
import math
import logging
import galsim
import random
import astropy.units as u
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def gal_flux(mag,exptime,image_size):
    tel_diam = 200
    lam_wid = 2500
    w = 5500 * u.AA
    flux_unit = (mag*u.ABmag).to(u.photon/u.s/u.cm**2/u.AA, u.spectral_density(w))
    ne = (flux_unit.value) * (np.pi*(0.5*tel_diam)**2)*exptime*lam_wid*gain
    flux = ne
    # sky background
    sky_mag = 23
    sky_flux = (sky_mag*u.ABmag).to(u.photon/u.s/u.cm**2/u.AA, u.spectral_density(w))
    sky_ne = (sky_flux.value) * (np.pi*(0.5*tel_diam)**2)*exptime*lam_wid*gain
    sky_pixel = sky_ne/image_size**2
    return flux,sky_pixel

def image(re,pixel_scale):
    image_size = int(5*re/pixel_scale+0.5)
    return image_size

def main(argv):
    psf_sigma = 0.2     # arcsec
    random_seed = 1314662
    rng = galsim.BaseDeviate(random_seed+1)
    #wcs_g1 = -0.02         
    #wcs_g2 = 0.01 
    
    # simple profile
    gal = galsim.Sersic(n, half_light_radius=re)
    gal_shape = galsim.Shear(q=q, beta=beta*galsim.degrees)
    gal = gal.withFlux(flux)
    gal = gal.shear(gal_shape)

    big_fft_params = galsim.GSParams(maximum_fft_size=12288)
    
    # psf
    psf = galsim.Gaussian(flux=1., sigma=psf_sigma) # PSF flux should always = 1
    final = galsim.Convolve([psf, gal],gsparams=big_fft_params)
    #wcs = galsim.ShearWCS(scale=pixel_scale, shear=galsim.Shear(g1=wcs_g1, g2=wcs_g2))
    
    # final image
    image = galsim.Image(image_size, image_size, scale=pixel_scale)
    final.drawImage(image=image)

    # add noise
    noise = galsim.CCDNoise(rng, gain=gain, read_noise=renoise, sky_level=sky_level_pixel)
    image.addNoise(noise)

    sn_pixel = []
    for x in range(0,image_size):
        for y in range(0,image_size):
            if math.sqrt((x-image_size/2)**2+(y-image_size/2)**2)<=(re/pixel_scale):
                if image[x,y]<0:
                    image[x,y]=0
                else:
                    sn_pixel.append(np.array(image[x,y])*exptime/
                                    math.sqrt((math.sqrt(np.array(image[x,y])*exptime))**2
                                              +renoise**2))
    sn_median = np.median(sn_pixel)
   
    # save file
    if sn_median>=10:
        path = '/home/zqw/data/data/'
        file_name =  str(i) + '.fits'
        file = os.path.join(path, file_name)
        image.write(file)
    '''fits.setval(file, 'EXPTIME', value='150')
    fits.setval(file, 'GAIN', value='1.0')
    fits.setval(file, 'RDNOISE', value='5')
    '''
    return sn_median

In [ ]:
exptime = 150
pixel_scale = 0.074
gain = 1
renoise = 5 
i = 0
f=open('/home/zqw/data/data.txt',mode='w')
for i in range(1,50001):
    n = random.uniform(0.5,6.1)
    re = random.uniform(0.5,3.51)
    q = random.uniform(0.2,0.81)
    mag = random.uniform(16,23.1)
    beta = random.randrange(0,180,20)
    image_size = image(re,pixel_scale)
    flux = gal_flux(mag,exptime,image_size)[0]
    sky_level_pixel = gal_flux(mag,exptime,image_size)[1]
    if __name__ == "__main__":  
        sn_median = main(sys.argv)
    f.writelines(str(i)+' '+str(n)+' '+str(re)+' '+str(q)+' '+
                 str(mag)+' '+str(beta)+' '+str(sn_median)+'\n')
f.close()